#### Evaluate Script of Trained Models

In [1]:
from run_multimodal_time_series import *

In [7]:
# loading model from saved model.
model = MultimodalEmotionPrediction()
model.load_state_dict(torch.load("../2021-07-03-run-1/best_ccc_pytorch_model.bin")["model"])
_ = model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing LinguisticEncoderBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.word_embeddings.weight']
- This IS expected if you are initializing LinguisticEncoderBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LinguisticEncoderBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LinguisticEncoderBERT were not initialized from the model che

In [8]:
tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-uncased",
    use_fast=False,
    cache_dir="../.huggingface_cache/"
)

In [10]:
# Loading all the data partitions.
data_dir = "../../SENDv1-data/"
train_modalities_data_dir = os.path.join(data_dir, "features/Train/")
train_target_data_dir = os.path.join(data_dir, "ratings/Train")
train_SEND_features = preprocess_SEND_files(
    train_modalities_data_dir,
    train_target_data_dir,
    linguistic_tokenizer=tokenizer,
    max_number_of_file=-1
)

dev_modalities_data_dir = os.path.join(data_dir, "features/Valid/")
dev_target_data_dir = os.path.join(data_dir, "ratings/Valid")
dev_SEND_features = preprocess_SEND_files(
    dev_modalities_data_dir,
    dev_target_data_dir,
    linguistic_tokenizer=tokenizer,
    max_number_of_file=-1
)

test_modalities_data_dir = os.path.join(data_dir, "features/Test/")
test_target_data_dir = os.path.join(data_dir, "ratings/Test")
test_SEND_features = preprocess_SEND_files(
    test_modalities_data_dir,
    test_target_data_dir,
    linguistic_tokenizer=tokenizer,
    max_number_of_file=-1
)

In [12]:
# Put dataset into correct format.
train_video_id = [video_struct["video_id"] for video_struct in train_SEND_features]
train_input_a_feature = torch.stack([video_struct["a_feature"] for video_struct in train_SEND_features]).float()
train_input_l_feature = torch.stack([video_struct["l_feature"] for video_struct in train_SEND_features])
train_input_l_mask = torch.stack([video_struct["l_mask"] for video_struct in train_SEND_features])
train_input_l_segment_ids = torch.stack([video_struct["l_segment_ids"] for video_struct in train_SEND_features])
train_input_v_feature = torch.stack([video_struct["v_feature"] for video_struct in train_SEND_features]).float()
train_rating_labels = torch.stack([video_struct["rating"] for video_struct in train_SEND_features]).float()
train_seq_lens = torch.tensor([[video_struct["seq_len"]] for video_struct in train_SEND_features]).float()
train_input_mask = torch.stack([video_struct["input_mask"] for video_struct in train_SEND_features])
train_data = TensorDataset(
    train_input_a_feature, 
    train_input_l_feature, train_input_l_mask, train_input_l_segment_ids,
    train_input_v_feature, train_rating_labels, train_seq_lens, train_input_mask
)
train_dataloader = DataLoader(train_data, batch_size=1, shuffle=False)

dev_video_id = [video_struct["video_id"] for video_struct in dev_SEND_features]
dev_input_a_feature = torch.stack([video_struct["a_feature"] for video_struct in dev_SEND_features]).float()
dev_input_l_feature = torch.stack([video_struct["l_feature"] for video_struct in dev_SEND_features])
dev_input_l_mask = torch.stack([video_struct["l_mask"] for video_struct in dev_SEND_features])
dev_input_l_segment_ids = torch.stack([video_struct["l_segment_ids"] for video_struct in dev_SEND_features])
dev_input_v_feature = torch.stack([video_struct["v_feature"] for video_struct in dev_SEND_features]).float()
dev_rating_labels = torch.stack([video_struct["rating"] for video_struct in dev_SEND_features]).float()
dev_seq_lens = torch.tensor([[video_struct["seq_len"]] for video_struct in dev_SEND_features]).float()
dev_input_mask = torch.stack([video_struct["input_mask"] for video_struct in dev_SEND_features])
dev_data = TensorDataset(
    dev_input_a_feature, 
    dev_input_l_feature, dev_input_l_mask, dev_input_l_segment_ids,
    dev_input_v_feature, dev_rating_labels, dev_seq_lens, dev_input_mask
)
dev_dataloader = DataLoader(dev_data, batch_size=1, shuffle=False)

test_video_id = [video_struct["video_id"] for video_struct in test_SEND_features]
test_input_a_feature = torch.stack([video_struct["a_feature"] for video_struct in test_SEND_features]).float()
test_input_l_feature = torch.stack([video_struct["l_feature"] for video_struct in test_SEND_features])
test_input_l_mask = torch.stack([video_struct["l_mask"] for video_struct in test_SEND_features])
test_input_l_segment_ids = torch.stack([video_struct["l_segment_ids"] for video_struct in test_SEND_features])
test_input_v_feature = torch.stack([video_struct["v_feature"] for video_struct in test_SEND_features]).float()
test_rating_labels = torch.stack([video_struct["rating"] for video_struct in test_SEND_features]).float()
test_seq_lens = torch.tensor([[video_struct["seq_len"]] for video_struct in test_SEND_features]).float()
test_input_mask = torch.stack([video_struct["input_mask"] for video_struct in test_SEND_features])
test_data = TensorDataset(
    test_input_a_feature, 
    test_input_l_feature, test_input_l_mask, test_input_l_segment_ids,
    test_input_v_feature, test_rating_labels, test_seq_lens, test_input_mask
)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

In [35]:
def evaluate_ablation(
    video_id, dataloader, model,
):
    video_index = 0
    for step, batch in enumerate(dataloader):
        vid_id = video_id[video_index]
        print(f"analyzing ablation studies on video_id={vid_id}")
        input_a_feature, input_l_feature, input_l_mask, input_l_segment_ids, \
            input_v_feature, rating_labels, seq_lens, input_mask = batch
        _, output = \
            model(input_a_feature, input_l_feature, input_l_mask, input_l_segment_ids,
                  input_v_feature, rating_labels, input_mask)
        seq_l = int(seq_lens[0].tolist()[0])
        pred = output[0][:seq_l].cpu().detach().numpy()
        true = rating_labels[0][:seq_l].cpu().detach().numpy()
        ccc = eval_ccc(pred, true)
        print(ccc)
        video_index += 1

In [36]:
_ = evaluate_ablation(train_video_id, train_dataloader, model)

analyzing ablation studies on video_id=ID162_vid4
-0.029260457911087293
analyzing ablation studies on video_id=ID167_vid2
-0.005360279659286024
analyzing ablation studies on video_id=ID129_vid2
-0.03959747014006752
analyzing ablation studies on video_id=ID156_vid1
0.06318631069985421
analyzing ablation studies on video_id=ID178_vid5
0.029169752435024475
analyzing ablation studies on video_id=ID118_vid1
-0.0017222610476933103
analyzing ablation studies on video_id=ID172_vid2
0.06498348697774153
analyzing ablation studies on video_id=ID142_vid3
0.07338304443307371
analyzing ablation studies on video_id=ID180_vid2
-0.016966083807458465
analyzing ablation studies on video_id=ID147_vid5
0.08804539993842923
analyzing ablation studies on video_id=ID113_vid4
-0.005784707516809227
analyzing ablation studies on video_id=ID171_vid2
-0.0014075871238926902
analyzing ablation studies on video_id=ID120_vid3
-0.028991286528435287
analyzing ablation studies on video_id=ID179_vid1
-0.01767528345994331
a